#Importing libraries

In [ ]:
import time
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn import svm
from sklearn.neural_network import MLPRegressor
!pip install GmdhPy
from gmdhpy.gmdh import MultilayerGMDH
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
!pip install SALib
from SALib.sample import fast_sampler
from SALib.analyze import fast

#preprocessing

In [ ]:
#importing data
df = pd.read_csv('data.csv')
df = df.sample(frac=1)
inputs = df.iloc[:,:6]
outputs = df.iloc[:,6:7]
#scaling features
scaler = StandardScaler()
scaler.fit(inputs)
scaled_inputs = scaler.transform(inputs)
#training/testing
X_train, X_test, y_train, y_test = train_test_split(scaled_inputs, outputs, 
                                                    shuffle=False,
                                                    test_size=0.15)


#SVM

In [ ]:
#hyperparameters_optimization
kernels = ['rbf', 'poly', 'sigmoid']
gammas = ['auto', 'scale']
Cs = [1, 9]
epsilons = [0.01, 0.1]
val=np.empty((24,2))
i=0
x = 0.0
for kernel in kernels:
    for gamma in gammas:
       for C in Cs:
         for epsilon in epsilons:
           start_time = time.time()
           regr = svm.SVR(kernel=kernel, gamma=gamma, C=C, epsilon=epsilon)
           cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
           n_scores = cross_val_score(regr, X_train, y_train, scoring='r2', cv=cv, n_jobs=-1, error_score='raise')
           valdiation_r2 = mean(n_scores) / std(n_scores)
           t = time.time()-start_time
           print(valdiation_r2, t)
           val[i:i+1,:] = [valdiation_r2, t]
           i+=1
           if valdiation_r2 > x:
            x = valdiation_r2
            a = kernel
            b = gamma
            c = C
            d = epsilon
np.savetxt('val.csv', val, delimiter =',')
a, b, c, d

In [ ]:
#model_training
t1 = time.time()
regr = svm.SVR(kernel='rbf', gamma='auto', C=9, epsilon = 0.01)
regr.fit(X_train, y_train)
t = -t1 + time.time()
#model_testing
test_r2 = regr.score(X_test, y_test)
test_mean_square_error = mean_squared_error(y_test, regr.predict(X_test))
test_r2, test_mean_square_error, t

#ANN

In [ ]:
#hyperparameters_optimization
hidden_layer_sizes = [(50,), (150,), (20,20)]
activations = ['relu', 'logistic', 'tanh']
alphas = [0.1, 1.0, 10.0]
x = 0.0
val=np.empty((27,2))
i=0
for hidden_layer_size in hidden_layer_sizes:
  for activation in activations:
      for alpha in alphas:
        start_time = time.time()
        ann = MLPRegressor(hidden_layer_sizes=hidden_layer_size,
                   activation=activation,
                   solver = 'lbfgs',
                   alpha = alpha)
        cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
        n_scores = cross_val_score(ann, X_train, y_train, scoring='r2', cv=cv, n_jobs=-1, error_score='raise')
        #print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
        valdiation_r2 = mean(n_scores) / std(n_scores)
        t = time.time()-start_time
        val[i:i+1,:] = [valdiation_r2, t]
        i+=1
        print(valdiation_r2, t)

        if valdiation_r2 > x:
          x = valdiation_r2
          a = hidden_layer_size
          b = activation
          c = alpha
np.savetxt('val.csv', val, delimiter =',')
a, b, c


In [ ]:
#model_training
t1 = time.time()
ann = MLPRegressor(hidden_layer_sizes=(50,),
                   solver = 'lbfgs',
                   max_iter=10000,
                   early_stopping=True,
                   alpha = 0.1
ann.fit(X_train, y_train)
t = -t1 + time.time()
#model_testing
test_r2 = ann.score(X_test, y_test)
test_mean_square_error = mean_squared_error(y_test, ann.predict(X_test))
test_r2, test_mean_square_error, t

#GMDH

In [ ]:
#hyperparameters_optimization
ref_functions = ['linear_cov', 'quadratic', 'cubic']
criterion_types = ['test', 'bias']
layer_err_criterions = ['avg', 'top']

for ref_function in ref_functions:
  for criterion_type in criterion_types:
    for layer_err_criterion in layer_err_criterions:
      gmdh = MultilayerGMDH(normalize=False,
                            ref_functions=ref_function,
                            criterion_type=criterion_type,
                            layer_err_criterion=layer_err_criterion)

      gmdh.fit(X_train, y_train.to_numpy())
      training_r2 = r2_score(y_test, gmdh.predict(X_test))


      if training_r2 > x:
          x = training_r2
          a = ref_function
          b = criterion_type
          c = layer_err_criterion
a,b,c      


In [ ]:
#model_training
t1 = time.time()
gmdh = MultilayerGMDH(normalize=False,
                      ref_functions=('quadratic','cubic', 'linear_cov'),
                      criterion_type='test',
                      layer_err_criterion='avg')
gmdh.fit(X_train.to_numpy(), y_train.to_numpy())
t = -t1 + time.time()
#model_testing
test_r2 = r2_score(y_test, gmdh.predict(X_test.to_numpy()))
test_mean_square_error = mean_squared_error(y_test, gmdh.predict(X_test.to_numpy()))
test_r2, test_mean_square_error, t

#XGBoost regression



In [ ]:
#hyperparameters_optimization
losses = ['ls', 'lad']
n_estimators = [100, 500]
max_features = [2, 3]
max_depths = [2, 3, 4]
x=0.0
val=np.empty((24,2))
i=0
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
for loss in losses:
  for n_estimator in n_estimators:
    for max_feature in max_features:
      for max_depth in max_depths:
        t1 = time.time()
        xgb = GradientBoostingRegressor(loss=loss,
                                        n_estimators=n_estimator,
                                        max_features=max_feature,
                                        max_depth=max_depth)
        n_scores = cross_val_score(xgb, X_train, y_train, scoring='r2', cv=cv, n_jobs=-1, error_score='raise')
        #print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
        valdiation_r2 = mean(n_scores) / std(n_scores)
        t = time.time() - t1
        val[i:i+1,:] = [valdiation_r2, t]
        i+=1
        print(valdiation_r2,t)

        if valdiation_r2 > x:
          x = valdiation_r2
          a = loss
          b = n_estimator
          c = max_feature
          d = max_depth
np.savetxt('val.csv', val, delimiter =',')
a, b, c, d

In [ ]:
#model_training
t1 = time.time()
xgb = GradientBoostingRegressor(loss='ls',
                                n_estimators=500,
                                max_features=3,
                                subsample=1)
xgb.fit(X_train, y_train)
t= -t1 + time.time()
#model_testing
test_r2 = xgb.score(X_test, y_test)
from sklearn.metrics import mean_squared_error
test_mean_square_error = mean_squared_error(y_test, xgb.predict(X_test))
test_r2, test_mean_square_error, t

## Senstivity Analysis

In [ ]:
problem = {'num_vars': 6,
           'names': ['T', 'Bu', 'T_irr', 'D', 'T_ann', 'O/M'],
           'bounds': [[300, 3200],
                     [0, 100],
                     [0, 3200],
                      [80, 100],
                      [300, 2000],
                      [2.0, 2.1]]
           }
param_values = fast_sampler.sample(problem, 1000000)
param_values = scaler.transform(param_values)
Y = xgb.predict(param_values)
Si = fast.analyze(problem, Y, print_to_console=True)